# Apply Photometric Classification To Fit Results

This notebook applies the photometric classification method from González-Gaitan 2014 to SDSS, DES, and CSP light-curve fits. Results are then used to analyze properties of peculiar supernovae.

In [ ]:
import numpy as np
import pandas as pd

from astropy.table import Table
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.palettes import Category20


## Initial FOM optimization

In [ ]:
csp_spec_class = Table.read('./csp_classifications.txt', format='ascii')
csp_spec_class.keep_columns(['SN', 'Type'])
csp_spec_class.show_in_notebook(display_length=10)

# Remove rows without data or targets with inconclusive classifications
csp_spec_class.remove_rows([0, 1])
is_data = np.logical_not(csp_spec_class['Type'] == '        ')
csp_spec_class = csp_spec_class[is_data]

csp_spec_class.show_in_notebook(display_length=10)


In [ ]:
def calc_fom(num_tot, num_true, num_false):
    """Calculate the figure of merrit for a set of classifications

    FOM = (num_true / num_tot) X (num_true / (num_true + num_false))
    
    args:
        num_tot   (float): Number of correct classifications for a given type
        num_true  (float): Total input number of that type
        num_false (float): Number of objects falsely classified for that type
        
    Returns:
        The figure of merit value
    """
    
    return (num_true / num_tot) * (num_true / (num_true + num_false))


## Combine summary tables from each fit run

In [ ]:
# Read in data from each fit run
data_tables = []
for fit_run in ('snia_ug', 'snia_riz', '91bg_ug', '91bg_riz'):
    sdss_path = '../sncosmo_fits/sdss_results/{}.csv'.format(fit_run)
    des_path = '../sncosmo_fits/des_results/{}.csv'.format(fit_run)
    
    sdss_data = pd.read_csv(sdss_path, index_col=0)
    des_data = pd.read_csv(des_path, index_col=0)
    
    sdss_data = sdss_data.rename(columns={
        'num_points_sdssu': 'num_points_u', 
        'num_points_sdssg': 'num_points_g',
        'num_points_sdssr': 'num_points_r',
        'num_points_sdssi': 'num_points_i',
        'num_points_sdssz': 'num_points_z'
    })
    
    des_data = des_data.rename(columns={
        'num_points_desu': 'num_points_u', 
        'num_points_desg': 'num_points_g',
        'num_points_desr': 'num_points_r',
        'num_points_desi': 'num_points_i',
        'num_points_desz': 'num_points_z'
    })
    
    
    combined_data = pd.concat([sdss_data, des_data], sort=False)
    combined_data.fillna(value={'class': 'DES'}, inplace=True)
    data_tables.append(combined_data)
    

# Combine data tables for snia and 91bg templates
combined_snia = data_tables[0].join(data_tables[1], lsuffix='_ug', rsuffix='_riz')
combined_snia['chi_norm_ug'] = combined_snia['chi_ug'] / combined_snia['dof_ug']
combined_snia['chi_norm_riz'] = combined_snia['chi_riz'] / combined_snia['dof_riz']

combined_91bg = data_tables[2].join(data_tables[3], lsuffix='_ug', rsuffix='_riz')
combined_91bg['chi_norm_ug'] = combined_91bg['chi_ug'] / combined_91bg['dof_ug']
combined_91bg['chi_norm_riz'] = combined_91bg['chi_riz'] / combined_91bg['dof_riz']

# Calculate chi-squared differences
chi_squared_values = pd.DataFrame({
    'blue': combined_snia['chi_norm_ug'] - combined_91bg['chi_norm_ug'],
    'red': combined_snia['chi_norm_riz'] - combined_91bg['chi_norm_riz'],
    'class': combined_snia['class_ug']
})


## Preliminary Chi-squared Comparison

In [ ]:
fig = figure(plot_width=600, plot_height=400, 
             tools=['save', 'pan', 'box_zoom', 'reset', 'wheel_zoom'])

class_set = sorted(set(chi_squared_values['class']))
for classification, color in zip(class_set, Category20[20]):
    indices = chi_squared_values['class'] == classification
    fig.circle(
        chi_squared_values['blue'][indices], 
        chi_squared_values['red'][indices], 
        color=color, 
        legend=classification
    )

# Lagend and labels
fig.legend.click_policy = "hide"
new_legend = fig.legend[0]
fig.legend[0].plot = None
fig.add_layout(new_legend, 'left')
fig.yaxis.axis_label = "Chi2 Ia - Chi2 91bg (riz)"
fig.xaxis.axis_label = "Chi2 Ia - Chi2 91bg (ug)"

output_notebook()
show(fig)
